# Section 1 Libraries and data
## About ARIMA:
- SARIMAX comes from ARIMA (Seasonal)
    -  It captures the seasonal patterns and variations that occur at fixed intervals, such as daily, weekly, monthly, or yearly patterns
- X (exogenous variables)
    - Exogenous variables are additional time series or factors that are not directly related to the time series being forecasted but may impact its behavior. 
- Auto-Regressive Integrated Moving Average
- Auto-Regressive: The output is regressed on its own lagged values.
- Integrated: Number of times we need to do differencing to make our time series stationary
- Moving Average: Instead of using the past values, the MA model uses past forecast error lags to predict future values.

In [2]:
#!pip install pmdarima

In [1]:
#libraries
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

from sklearn.model_selection import ParameterGrid # performing grid search or generating all possible combinations of hyperparameter

import pmdarima as pm # Pyramid ARIMA
from pmdarima import model_selection # functions for model selection and cross-validation

In [2]:
#load the data
#YYYY-MM-DD
df = pd.read_csv('nyc_data.csv', index_col = 0, parse_dates = True)
df.head()

,Demand,Easter,Thanksgiving,Christmas,Temperature,Marketing
Date,,,,,,
2015-01-01,720.000885,0,0,0,3.68,41.305
2015-01-02,581.276773,0,0,0,4.73,131.574
2015-01-03,754.117039,0,0,0,7.23,162.700
2015-01-04,622.252774,0,0,0,10.96,160.281
2015-01-05,785.373319,0,0,0,6.92,51.077


In [3]:
#Rename variable
df = df.rename(columns = {'Demand': 'y'})
df.head(0)

,y,Easter,Thanksgiving,Christmas,Temperature,Marketing
Date,,,,,,


In [4]:
#Extract regressors
X = df.iloc[:,1:]
X.head(0)

,Easter,Thanksgiving,Christmas,Temperature,Marketing
Date,,,,,


## AutoRegressive components(AR):
    Past values, the lags, contain information that help predict future values

- How to determine how many lags?
    - We'll perform parameter tuning.

## Integrating components(I):
    In SARIMAX, the "I" component refers to the integrated part, which represents the number of times differencing is applied to the time series. The integrated order, denoted as "d" in SARIMAX(p, d, q)(P, D, Q, s), specifies the number of differences needed to achieve stationarity.

# Section 2 Stationarity

- Dicky-Fuller test: If p-value is less than 0.05, time series is considered stationary.
- In SARIMAX modeling, stationarity is an important concept that needs to be considered. A stationary time series is one where the statistical properties, such as the mean, variance, and autocorrelation, do not change over time. Stationarity is desirable in time series modeling because it simplifies the modeling process and allows for more reliable forecasts.
- You typically apply ***differencing*** to the time series data. Differencing is a process where you compute the differences between consecutive observations. 

## Dicky-Fuller test

In [5]:
#Test
from statsmodels.tsa.stattools import adfuller
pvalue = adfuller(x = df.y)[1] #check afuller() for more. Here, the second parameter is pvalue.

#condition to read test
if pvalue < 0.05:
    print(f"The Time Series is stationary. The p-value is {pvalue}")
else:
    print(f"The Time Series is not stationary. The p-value is {pvalue}")

The Time Series is not stationary. The p-value is 0.3767770707729106


In [6]:
df.y.diff()

Date
2015-01-01           NaN
2015-01-02   -138.724112
2015-01-03    172.840266
2015-01-04   -131.864264
2015-01-05    163.120544
                 ...    
2020-12-27   -227.008591
2020-12-28    312.136144
2020-12-29   -150.927771
2020-12-30     10.397644
2020-12-31    -73.629549
Name: y, Length: 2192, dtype: float64

## Differencing

In [7]:
#differencing
df.y.diff().dropna() #drop the first one

#Test
pvalue = adfuller(x = df.y.diff().dropna())[1]

#condition to read test
if pvalue < 0.05:
    print(f"The Time Series is stationary. The p-value is {pvalue}")
else:
    print(f"The Time Series is not stationary. The p-value is {pvalue}")

The Time Series is stationary. The p-value is 3.355773945637397e-22


# Section 3 SARIMAX

## Step 1 SARIMAX Model

In [8]:
#Model
#hourly: 24, daily: 7, weekly:52, monthly:12, quarterly:4
model = pm.ARIMA(order = (1,1,1),
                 seasonal_order = (1,1,1, 7),
                 X = X,
                 suppress_warning = True,
                 force_stationarity = False)

In [9]:
df.shape[0]

2192

### CV Strategy: RollingForecastCV

In [10]:
#Define Cross-validation strategy
cv = model_selection.RollingForecastCV(h = 31, # horizon = each test set size is 31 days
                                       step = 16, #training size will move forward by 16 samples
                                       initial = df.shape[0] - 180) #initial = train size
cv

RollingForecastCV(h=31, initial=2012, step=16)

For RollingForecastCV, 

    number_of_folds = (total_samples - initial_size - horizon) / step + 1

    number_of_folds = (df.shape[0] - (df.shape[0] - 180) - 31) / 16 + 1
                   = (180 - 31) / 16 + 1
                   = 149 / 16 + 1
                   = 9.3125 + 1
                   = 10.3125
                   
So that we'll have 10 folds and 10 cv_score in the array.

### Overview of the validation strategies 

To determine the type of cross-validation strategy you are using, you need to look at the specific function or class you are using to perform cross-validation. Here are some common cross-validation strategies and how you can identify them:

1. `KFold`: If you are using the `KFold` class from the `sklearn.model_selection` module, it means you are using k-fold cross-validation. This strategy splits the dataset into k equally sized folds and iterates through them as the training and testing sets.

2. `StratifiedKFold`: If you are using the `StratifiedKFold` class from the `sklearn.model_selection` module, it means you are using stratified k-fold cross-validation. This strategy preserves the class distribution in each fold, which is useful for imbalanced datasets.

3. `ShuffleSplit`: If you are using the `ShuffleSplit` class from the `sklearn.model_selection` module, it means you are using shuffle and split cross-validation. This strategy randomly shuffles the dataset and splits it into training and testing sets multiple times.

4. `TimeSeriesSplit`: If you are using the `TimeSeriesSplit` class from the `sklearn.model_selection` module, it means you are using time series cross-validation. This strategy splits the dataset into sequential folds, ensuring that the training sets come before the testing sets in terms of time.

5. `RollingForecastCV`: If you are using the `RollingForecastCV` class from the `pmdarima.model_selection` module, it specifically refers to rolling forecast cross-validation. This strategy is suitable for time series data and involves creating overlapping training and testing sets as the window moves forward in time.

To identify the cross-validation strategy, check the specific function or class you are using and refer to the documentation for more details on how it performs the data splitting and iteration.


### Choice of cross-validation strategy

The choice of cross-validation strategy depends on several factors, including the nature of your data, the problem you are solving, and the specific requirements of your analysis. Here are some considerations to help you determine which cross-validation strategy to use:

1. **Dataset characteristics**: Consider the properties of your dataset, such as whether it is a time series data or if it has class imbalance. Time series data often requires specialized time-based cross-validation strategies like rolling forecast or time series split. For imbalanced datasets, you may consider using stratified cross-validation to ensure representative class distributions in each fold.

2. **Data size**: If you have a large dataset, you may opt for computationally efficient strategies like k-fold cross-validation. However, if your dataset is small, you might choose leave-one-out cross-validation or stratified k-fold to maximize the use of available data for training and testing.

3. **Goal of analysis**: Consider the specific goal of your analysis. For example, if you are interested in evaluating the performance of a model and obtaining reliable estimates of its performance, you might choose a cross-validation strategy that provides a robust estimate of model performance, such as k-fold cross-validation. If your goal is to assess model performance on future unseen data, time series cross-validation or rolling forecast cross-validation would be more appropriate.

4. **Available computational resources**: Some cross-validation strategies, such as leave-one-out or repeated k-fold cross-validation, can be computationally expensive, especially for large datasets. Consider the available computational resources and time constraints when selecting a cross-validation strategy.

5. **Domain-specific considerations**: Depending on the specific requirements or conventions in your domain, there may be recommended or commonly used cross-validation strategies. Consult domain experts or related research papers to understand if there are any specific guidelines or standards for cross-validation in your field.

Ultimately, it's important to understand the characteristics of different cross-validation strategies and select the one that best suits your data, problem, and evaluation requirements. It may also be helpful to experiment with multiple strategies and compare their results to gain insights into the stability and performance of your models.

### Auto_ARIMA and ARIMA
The main difference between using `auto_arima` and manually specifying the ARIMA model parameters is the approach taken to determine the optimal model. 

In `auto_arima`, the function automatically searches and evaluates different combinations of ARIMA orders using a stepwise algorithm. It considers various combinations of parameters and selects the model that minimizes a given criterion, such as the AIC or BIC. This automated process saves you the effort of manually tuning the parameters and allows you to quickly obtain a reasonable model.

On the other hand, when you manually specify the ARIMA model parameters, as in your code snippet, you have control over the specific orders you want to use. You can set the orders based on your prior knowledge or domain expertise. This approach gives you more flexibility and allows you to directly define the model structure you believe is appropriate for your data.

After fitting the initial model using `pm.ARIMA`, you can perform tuning by evaluating the model's performance and adjusting the parameters accordingly. This can involve exploring different combinations of orders, such as changing the values of `p`, `d`, `q`, `P`, `D`, and `Q`, to find the best-fitting model. 

Both approaches have their advantages and considerations. `auto_arima` is convenient and automates the parameter selection process, especially if you don't have prior knowledge about the data. Manually specifying the parameters allows for more customization and domain-specific modeling choices. Ultimately, the choice between the two depends on your specific needs, time constraints, and the level of control you want over the model selection process.


## Step 2 CV score

In [10]:
#cv score
cv_score = model_selection.cross_val_score(model,
                                           y = df.y, #target variable that trying to predict
                                           scoring = 'mean_squared_error', #evaluation metrics
                                           cv = cv, # cross-validation strategy that I wanna use
                                           verbose = 2, #visualize the performance report, can be set to 0,1,2
                                           error_score = 1000000000000000) #assigning a high penalty for any error that occurs

[CV] fold=0 ..........................................................


/Users/aurorafu/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
/Users/aurorafu/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
/Users/aurorafu/opt/anaconda3/lib/python3.9/site-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/Users/aurorafu/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
/Users/aurorafu/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:471: Val

[CV] fold=1 ..........................................................
[CV] fold=2 ..........................................................


/Users/aurorafu/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
/Users/aurorafu/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)


[CV] fold=3 ..........................................................


...

/Users/aurorafu/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
/Users/aurorafu/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)


[CV] fold=8 ..........................................................


/Users/aurorafu/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
/Users/aurorafu/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)


[CV] fold=9 ..........................................................


/Users/aurorafu/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
/Users/aurorafu/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)


## Step 3 CV performance (RMSE)

In [12]:
cv_score

array([3183.63209778, 2396.51103667, 2298.8640604 , 2975.56555471,
       2775.78727568, 6167.79444007, 2192.04117443, 2710.19113386,
       5339.21319787, 5906.69125773])

In [14]:
#CV performance
error = np.sqrt(np.average(cv_score))
error

59.95522598506731

# Section 4 Parameter Tuning

## ParameterGrid (Grid Search)

### Determine ARIMA parameters
Determining the appropriate values for the ARIMA model parameters (`p`, `d`, `q`, `P`, `D`, `Q`) in your case can be approached in several ways. Here are a few considerations:

1. Prior knowledge or domain expertise: If you have prior knowledge or domain expertise about the data and the factors that might influence it, you can use that knowledge to guide your parameter selection. For example, if you believe that the data has a strong seasonal pattern with a weekly cycle, you can set `P` and `Q` to be non-zero values (e.g., 1) to capture the seasonal component.

2. Exploratory data analysis: Analyze the patterns and characteristics of your data through visualizations and statistical analysis. Look for any evident seasonal patterns, trends, or irregularities. This analysis can provide insights into the appropriate values for the ARIMA parameters. For example, if you observe a clear weekly seasonality in your data, you can set `P` and `Q` to capture this seasonality.

3. Grid search or automated approaches: You can use grid search or automated algorithms, such as `auto_arima` from the `pmdarima` library, to explore different combinations of parameter values and select the best-fitting model based on a predefined criterion, such as AIC or BIC. These methods can help you systematically evaluate multiple parameter combinations and choose the one that yields the best performance.

4. Model evaluation and iteration: Fit the ARIMA model with different parameter combinations and evaluate their performance using appropriate evaluation metrics (e.g., RMSE, MAPE). Compare the performance of different models and iterate by adjusting the parameter values to improve the model's accuracy. Repeat this process until you find a satisfactory model.

Remember that selecting the optimal ARIMA parameters is an iterative process that often requires experimentation, evaluation, and refinement. It's important to consider the characteristics of your data, any domain knowledge, and the evaluation metrics to make informed decisions about the parameter values.

### Check EDA
Seasonal decomposition can be a helpful tool in determining the appropriate values for the ARIMA model parameters. By decomposing your time series into its underlying components (trend, seasonality, and residual), you can gain insights into the patterns and characteristics of your data.

The four graphs generated from seasonal decomposition are typically:
1. Observed: This shows the original time series data, allowing you to visualize any overall trend or irregularities in the data.
2. Trend: This represents the long-term, smooth variation in the data, which helps identify any persistent upward or downward movement.
3. Seasonal: This reveals the repetitive, seasonal patterns or cycles in the data, such as daily, weekly, or monthly patterns.
4. Residual: This displays the remaining variation in the data that is not explained by the trend or seasonality. It represents the random or irregular component of the time series.

Analyzing these decomposition graphs can provide insights into the values of the seasonal parameters (`P` and `Q`). Here's how they can help:

1. Trend: If you observe a significant upward or downward trend in the data, you might need to include a differencing term (`d` or `D`) to make the data stationary. A differencing term can help remove any trends and make the data more suitable for modeling.

2. Seasonal: If you notice clear and consistent patterns in the seasonal component graph, it suggests the presence of seasonality in the data. The period of the seasonality can help determine the values of the seasonal parameters (`P` and `Q`). For example, if you see a repeating pattern every 7 days, you can set `P` and `Q` accordingly to capture the weekly seasonality.

3. Residual: Examining the residual component can provide insights into the presence of any remaining patterns or irregularities. If the residual component exhibits significant patterns or autocorrelation, it suggests that the current model might not be capturing all the underlying patterns in the data. Adjusting the ARIMA parameters (`p`, `q`, `P`, `Q`) can help improve the model's ability to capture the remaining variation.

Overall, seasonal decomposition can serve as a visual guide to identify the presence and characteristics of trend and seasonality in your data, which can guide your selection of ARIMA parameters. However, it's important to combine these visual observations with statistical analysis and evaluation metrics to make a well-informed decision about the parameter values.


### Check Seasonal Decomposition
If you observe yearly patterns in your seasonal graph, it suggests that there is a seasonal component in your data that repeats on a yearly basis. To capture this seasonal pattern, you can consider including autoregressive (AR) and moving average (MA) terms with lag orders of 0, 1, and 2.

The autoregressive term (p) represents the dependence of the current observation on the previous observations, while the moving average term (q) represents the dependency on the previous forecast errors. By including lag orders of 0, 1, and 2, you allow the model to capture different levels of autocorrelation and moving average effects within the yearly pattern.

### Grid Search for AR

### Stationarity for I
In SARIMAX, the "I" component refers to the integrated part, which represents the number of times differencing is applied to the time series.

### Grid Search for MA

In [50]:
#Grid
param_grid = {'p': [0, 1, 2],
              'd': [1],
              'q': [0, 1, 2],
              'P': [0, 1, 2],
              'D': [1],
              'Q': [0, 1, 2]
             }
grid = ParameterGrid(param_grid)
len(list(grid))

81

## Paramter tuning

- Combing section 3 step 123

In [51]:
#Parameter tuning
rmse = []
i = 1
#Parameter loop
for params in grid:
    print(f" {i} / {len(list(grid))}")
    
    #Step 1 model
    model = pm.ARIMA(order = (params['p'],params['d'],params['q']),
                  seasonal_order = (params['P'],params['D'],params['Q'], 7),
                  X = X,
                  suppress_warning = True,
                  force_stationarity = False)
  
    #Step 2 CV
    cv = model_selection.RollingForecastCV(h = 31,
                                        step = 16,
                                        initial = df.shape[0] - 180)
    cv_score = model_selection.cross_val_score(model,
                                            y = df.y,
                                            scoring = 'mean_squared_error',
                                            cv = cv,
                                            verbose = 2,
                                            error_score = 1000000000000000)

    #Step 3 Error
    error = np.sqrt(np.average(cv_score))
    rmse.append(error)
    
    i += 1

 1 / 81
[CV] fold=0 ..........................................................
[CV] fold=1 ..........................................................
[CV] fold=2 ..........................................................
[CV] fold=3 ..........................................................
[CV] fold=4 ..........................................................


/Users/aurorafu/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
/Users/aurorafu/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
/Users/aurorafu/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
/Users/aurorafu/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
/Users/aurorafu/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: No freque

[CV] fold=5 ..........................................................
[CV] fold=6 ..........................................................
[CV] fold=7 ..........................................................
[CV] fold=8 ..........................................................
[CV] fold=9 ..........................................................


/Users/aurorafu/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
/Users/aurorafu/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
/Users/aurorafu/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
/Users/aurorafu/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
/Users/aurorafu/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: No freque

 2 / 81
[CV] fold=0 ..........................................................


...

[CV] fold=7 ..........................................................


/Users/aurorafu/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
/Users/aurorafu/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)


[CV] fold=8 ..........................................................


/Users/aurorafu/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
/Users/aurorafu/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)


ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [52]:
len(rmse)

69

In [55]:
min(rmse)

58.891396596206434

In [56]:
#Check the results
tuning_results = pd.DataFrame(grid)
tuning_results['rmse'] = np.nan
tuning_results.loc[:len(rmse)-1, 'rmse'] = rmse
tuning_results

,D,P,Q,d,p,q,rmse
0,1,0,0,1,0,0,220.335793
1,1,0,0,1,0,1,75.031928
2,1,0,0,1,0,2,75.118102
3,1,0,0,1,1,0,184.967356
4,1,0,0,1,1,1,74.820682
...,...,...,...,...,...,...,...
76,1,2,2,1,1,1,NaN
77,1,2,2,1,1,2,NaN
78,1,2,2,1,2,0,NaN
79,1,2,2,1,2,1,NaN


## Export best parameter

In [57]:
best_params = tuning_results[tuning_results.rmse == tuning_results.rmse.min()].transpose()
best_params

,51
D,1.000000
P,1.000000
Q,2.000000
d,1.000000
p,2.000000
q,0.000000
rmse,58.891397


In [58]:
#export best parameters
best_params.to_csv("best_params_sarimax.csv")

# Auto_ARIMA

In [17]:
train = df.y[:-180]
train

Date
2015-01-01    720.000885
2015-01-02    581.276773
2015-01-03    754.117039
2015-01-04    622.252774
2015-01-05    785.373319
                 ...    
2020-06-30    744.075309
2020-07-01    808.960496
2020-07-02    715.503976
2020-07-03    644.183690
2020-07-04    767.649479
Name: y, Length: 2012, dtype: float64

In [19]:
import numpy as np
import pmdarima as pm


# fit stepwise auto-ARIMA
arima = pm.auto_arima(train, start_p=1, start_q=1,
                             max_p=3, max_q=3, m=7,
                             start_P=0, seasonal=True,
                             d=1, D=1, trace=True,
                             error_action='ignore',  # don't want to know if an order does not work
                             suppress_warnings=True,  # don't want convergence warnings
                             stepwise=True)  # set to stepwise
arima.summary()

Performing stepwise search to minimize aic
 ARIMA(1,1,1)(0,1,1)[7]             : AIC=inf, Time=1.37 sec
 ARIMA(0,1,0)(0,1,0)[7]             : AIC=23928.163, Time=0.03 sec
 ARIMA(1,1,0)(1,1,0)[7]             : AIC=22835.262, Time=0.20 sec
 ARIMA(0,1,1)(0,1,1)[7]             : AIC=inf, Time=0.95 sec
 ARIMA(1,1,0)(0,1,0)[7]             : AIC=23365.416, Time=0.05 sec
 ARIMA(1,1,0)(2,1,0)[7]             : AIC=22583.502, Time=0.46 sec
 ARIMA(1,1,0)(2,1,1)[7]             : AIC=inf, Time=2.23 sec
 ARIMA(1,1,0)(1,1,1)[7]             : AIC=inf, Time=1.62 sec
 ARIMA(0,1,0)(2,1,0)[7]             : AIC=23130.597, Time=0.18 sec
 ARIMA(2,1,0)(2,1,0)[7]             : AIC=22330.563, Time=0.57 sec
 ARIMA(2,1,0)(1,1,0)[7]             : AIC=22569.522, Time=0.32 sec
 ARIMA(2,1,0)(2,1,1)[7]             : AIC=inf, Time=2.51 sec
 ARIMA(2,1,0)(1,1,1)[7]             : AIC=inf, Time=1.12 sec
 ARIMA(3,1,0)(2,1,0)[7]             : AIC=22217.780, Time=0.70 sec
 ARIMA(3,1,0)(1,1,0)[7]             : AIC=22459.534, Ti

<class 'statsmodels.iolib.summary.Summary'>
"""
                                     SARIMAX Results                                     
=========================================================================================
Dep. Variable:                                 y   No. Observations:                 2012
Model:             SARIMAX(3, 1, 0)x(2, 1, 0, 7)   Log Likelihood              -11102.890
Date:                           Wed, 28 Jun 2023   AIC                          22217.780
Time:                                   11:30:43   BIC                          22251.398
Sample:                               01-01-2015   HQIC                         22230.123
                                    - 07-04-2020                                         
Covariance Type:                             opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ar.L1         -0.7404      0.021    -35.716      0.000      -0.781      -0.700
ar.L2         -0.5014      0.023    -21.378      0.000      -0.547      -0.455
ar.L3         -0.2359      0.021    -11.268      0.000      -0.277      -0.195
ar.S.L7       -0.6568      0.019    -33.788      0.000      -0.695      -0.619
ar.S.L14      -0.3389      0.020    -17.363      0.000      -0.377      -0.301
sigma2      3792.2646    109.451     34.648      0.000    3577.745    4006.784
===================================================================================
Ljung-Box (L1) (Q):                   2.24   Jarque-Bera (JB):                19.58
Prob(Q):                              0.13   Prob(JB):                         0.00
Heteroskedasticity (H):               1.86   Skew:                            -0.04
Prob(H) (two-sided):                  0.00   Kurtosis:                         3.48
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

In [23]:
model = pm.ARIMA(order = (3,1,0),
                  seasonal_order = (2,1,0, 7),
                  X = X,
                  suppress_warning = True,
                  force_stationarity = False)
  

cv = model_selection.RollingForecastCV(h = 31,
                                        step = 16,
                                        initial = df.shape[0] - 180)
cv_score = model_selection.cross_val_score(model,
                                            y = df.y,
                                            scoring = 'mean_squared_error',
                                            cv = cv,
                                            verbose = 2,
                                            error_score = 1000000000000000)
error = np.sqrt(np.average(cv_score))
error

[CV] fold=0 ..........................................................


/Users/aurorafu/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
/Users/aurorafu/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)


[CV] fold=1 ..........................................................


/Users/aurorafu/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
/Users/aurorafu/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)


[CV] fold=2 ..........................................................


...

[CV] fold=8 ..........................................................


/Users/aurorafu/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
/Users/aurorafu/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)


[CV] fold=9 ..........................................................


/Users/aurorafu/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
/Users/aurorafu/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)


88.4425634150678